In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/Trendyol.zip

Archive:  /content/drive/MyDrive/Trendyol.zip
  inflating: supplier_return.csv     
  inflating: test_data.csv           
  inflating: transactions.csv        
  inflating: user_demographics.csv   
  inflating: products.csv            
  inflating: qa.csv                  
  inflating: reviews.csv             
  inflating: supplier_defective_return.csv  
  inflating: supplier_disputed_return.csv  


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
products = pd.read_csv(r"products.csv")
qa = pd.read_csv(r"qa.csv")
reviews = pd.read_csv(r"reviews.csv")
supplier_defective_return = pd.read_csv(r"supplier_defective_return.csv")
supplier_disputed_return = pd.read_csv(r"supplier_disputed_return.csv")
supplier_return = pd.read_csv(r"supplier_return.csv")
test_data = pd.read_csv(r"test_data.csv")
transactions = pd.read_csv(r"transactions.csv")
user_demographics = pd.read_csv(r"user_demographics.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
left_merged_reversed = pd.merge(transactions,products,how="left",on=["product_variant_id", "product_content_id"])
reviews = reviews.groupby(["product_content_id", "supplier_id"])["rate"].mean().to_frame()
merge_review = pd.merge(left_merged_reversed,reviews,how="left",on=["product_content_id", "supplier_id"])
user_id_gender = user_demographics.drop(["birth_date", "membership_date"], axis=1)
merge_demographics = pd.merge(merge_review,user_id_gender,how="left",on="user_id")


In [ ]:
del transactions
del reviews
del user_demographics
del products
del user_id_gender
del left_merged_reversed
del merge_review


In [ ]:
merge_return = pd.merge(merge_demographics,supplier_return,how="left",on = "supplier_id") 

In [ ]:
supplier_defective_return = supplier_defective_return.rename(columns = {"returnRate":"defective_returnRate"})
merge_defective_return = pd.merge(merge_return,supplier_defective_return,how="left",on = "supplier_id")

In [ ]:
del supplier_disputed_return["unresolved_claim"]
del supplier_disputed_return["unresolved_accepted_claim"]
del supplier_defective_return
del merge_return

In [ ]:
merge_disputed_return = pd.merge(merge_defective_return,supplier_disputed_return,how="left",on = "supplier_id")

In [ ]:
df = merge_disputed_return 
df.head()

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned,product_id,product_name,brand_id,brand_name,gender_id,gender_name,category_id,category_name,color_id,color_name,supplier_color_name,attributet_name,attribute_value,rate,gender,returnRate,defective_returnRate,total_claim,unresolvedclaim_percentage
0,2021-05-01 00:00:01.761000+00:00,3941815,0,4662,901280147,595758968,95503167,159147197,79.90,51.94,8.88,NaN,0.0,Sepette %35 İndirim,35.0,0,True,0.0,82687030,Manage Erkek T-shirt Beyaz St11te151,682,Slazenger,4294.0,Erkek,604,T-Shirt,6998.0,Beyaz,Beyaz,BEDEN,XL,3.25000,Bayan,0.098178,0.003045,16357.0,0.328947
1,2021-05-01 00:00:01.761000+00:00,3941815,0,163,901280149,595758968,36313798,64622938,129.95,59.97,9.40,NaN,0.0,Sepette %40 İndirim,40.0,0,True,0.0,30551316,Erkek Koyu Yeşil Slim Fit V Yaka T-Shirt,121,Pierre Cardin,4294.0,Erkek,604,T-Shirt,7012.0,Yeşil,VR079,BEDEN,XL,4.47619,Bayan,0.050737,0.000545,46699.0,0.000000
2,2021-05-01 00:00:01.761000+00:00,3941815,0,157306,901280145,595758968,36888320,65612791,59.99,49.99,10.67,NaN,0.0,NaN,NaN,0,True,0.0,30961805,Erkek MIGAYA 012208415760890000,381,Ltb,4294.0,Erkek,2949,Polo Yaka T-shirt,6999.0,Gri,GRİ_217,BEDEN,XL,NaN,Bayan,0.093355,0.000744,902.0,0.777778
3,2021-05-01 00:00:01.761000+00:00,3941815,0,163856,901280144,595758968,36888399,65613254,49.90,49.90,10.67,NaN,0.0,NaN,NaN,0,True,0.0,30961856,Erkek MICAFO 012208418960890000,381,Ltb,4294.0,Erkek,2949,Polo Yaka T-shirt,7006.0,Mor,MOR_11525,BEDEN,XL,NaN,Bayan,0.108296,0.000633,227.0,0.200000
4,2021-05-01 00:00:02.742000+00:00,15996105,1,968,901279297,595758987,96930601,161408176,89.99,71.99,9.40,NaN,0.0,Sepette %20 İndirim,20.0,0,False,0.0,82735052,Mavi Leopar Desenli Dokuma Elbise THMSS21EL3385,40,TRENDYOLMİLLA,4295.0,Kadın / Kız,1182,Elbise,7004.0,Mavi,Mavi,BEDEN,40,4.50000,Bayan,0.127110,0.002828,3269293.0,0.000000


In [ ]:
del df["order_date"]
del df["user_id"]
del df["supplier_id"]
del df["order_line_item_id"]
del df["order_parent_id"]
del df["product_content_id"]
del df["product_variant_id"]
del df["coupon_id"]
del df["promotion_name"]
del df["product_id"]
del df["product_name"]
del df["brand_id"]
del df["brand_name"] # Sonra bakılcak.
del df["gender_id"]
del df["category_id"]
del df["category_name"] # Sonra bakılcak.
del df["color_id"]
del df["supplier_color_name"] 
del df["attributet_name"] # Sonra Bakılcak.
del df["attribute_value"]

In [ ]:
df = df[:3000000]

In [ ]:
df.to_csv("df_12m.csv")

In [ ]:
df = df[:3000000]
df.shape

In [ ]:
is_returned = df["is_returned"]
del df["is_returned"]

In [ ]:
df["ship_cost"] = df["ship_cost"].fillna(0)

df["promotion_award_value"] = df["promotion_award_value"].fillna(0)

df["rate"] = df["rate"].fillna(0)

df["color_name"] = df["color_name"].fillna("yok")

df["unresolvedclaim_percentage"] = df["unresolvedclaim_percentage"].fillna(0)

df["total_claim"] = df["total_claim"].fillna(0)

df["defective_returnRate"] = df["defective_returnRate"].fillna(0)

df["returnRate"] = df["returnRate"].fillna(0)


In [ ]:
df.isnull().sum().sort_values(ascending = False)

In [ ]:
pip install category_encoders

In [ ]:
import category_encoders as ce
ohe = ce.OneHotEncoder(handle_unknown='value', use_cat_names=True)

OH_cols_train = pd.DataFrame(ohe.fit_transform(df[["gender_name", "color_name", "gender"]])) #One-Hot encoding applied to train dataset
OH_cols_train.index = df.index
df = df.drop(columns = ["gender_name", "color_name", "gender"]) #Dropping categorical columns
df = pd.concat([df,OH_cols_train],axis = 1) # Assembling df with encoded columns

In [ ]:
df["is_saved_card_trx"] = df["is_saved_card_trx"].astype(int)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, is_returned, test_size = 0.4, random_state = 31)

In [ ]:
print(len(is_returned[is_returned == 0]),len(is_returned[is_returned == 1]))

In [ ]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train, y_train)

In [ ]:
y_pred = xgb_cl.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="macro")

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
import pickle
with open("XGB_2.pk1", 'wb') as file:  
    pickle.dump(xgb_cl, file)